In [5]:
import urllib.request
import json
import dml
import prov.model
import datetime
import uuid
import geojson


def fetch_json(url, item):
    print("Downloading " + str(item) + " Dataset from: " + str(url))
    response = urllib.request.urlopen(url).read().decode("utf-8")
    r = json.loads(response)
    return r


def fetch_geojson(url, item):
    print("Downloading " + str(item) + " Dataset from: " + str(url))
    response = urllib.request.urlopen(url).read().decode("utf-8")
    r = geojson.loads(response)
    rdict = dict(r)
    rlist = rdict['features']
    return rlist


class get_data2(dml.Algorithm):
    contributor = 'jhs2018_rpm1995'
    reads = []
    writes = ['jhs2018_rpm1995.utilitybills']

    @staticmethod
    def execute(trial=False):
        # Retrieve datasets
        startTime = datetime.datetime.now()

        # Set up the database connection.
        client = dml.pymongo.MongoClient()
        repo = client.repo
        repo.authenticate('jhs2018_rpm1995', 'jhs2018_rpm1995')
        
        r = fetch_json('https://data.boston.gov/export/35f/ad2/35fad26c-1400-46b0-846c-3bb6ca8f74d0.json', "Utility Bills")
        repo.dropCollection("utilitybills")
        repo.createCollection("utilitybills")
        repo['jhs2018_rpm1995.utilitybills'].insert_many(r)
        
        endTime = datetime.datetime.now()

        return {"start": startTime, "end": endTime}
    
    @staticmethod
    def provenance(doc=prov.model.ProvDocument(), startTime=None, endTime=None):

            # Create the provenance document describing everything happening
            # in this script. Each run of the script will generate a new
            # document describing that invocation event.

        client = dml.pymongo.MongoClient()
        repo = client.repo
        repo.authenticate('jhs2018_rpm1995', 'jhs2018_rpm1995')
        doc.add_namespace('alg', 'http://datamechanics.io/algorithm/')  # The scripts are in <folder>#<filename> format.
        doc.add_namespace('dat', 'http://datamechanics.io/data/')  # The data sets are in <user>#<collection> format.
        doc.add_namespace('ont', 'http://datamechanics.io/ontology#')  # 'Extension', 'DataResource', 'DataSet',
        # 'Retrieval', 'Query', or 'Computation'.
        doc.add_namespace('log', 'http://datamechanics.io/log/')  # The event log.
        doc.add_namespace('bwod', 'https://data.boston.gov/dataset/city-of-boston-utility-data/')
        
        this_script = doc.agent('alg:jhs2018_rpm1995#getdata', {prov.model.PROV_TYPE: prov.model.PROV['SoftwareAgent'], 'ont:Extension': 'py'})
        
        
        resource_utility = doc.entity('bwod: utilitybills', {'prov:label': 'Utility Bills Boston Buildings', prov.model.PROV_TYPE:
                                                                                        'ont:DataResource',
                                                                                    'ont:Extension': 'json'})
        get_utility = doc.activity('log:uuid'+str(uuid.uuid4()), startTime, endTime)
        doc.wasAssociatedWith(get_utility, this_script)
        doc.usage(get_utility, resource_utlility, startTime, None, {prov.model.PROV_TYPE: 'ont:Retrieval'})
        
        
        utility = doc.entity('dat:jhs2018_rpm1995utilitybills',
                                {prov.model.PROV_LABEL: 'utility bills', prov.model.PROV_TYPE: 'ont:DataSet'})
        doc.wasAttributedTo(utility, this_script)
        doc.wasGeneratedBy(utility, get_utility, endTime)
        doc.wasDerivedFrom(utility, resource_utility, get_utility, get_utility, get_utility)

        repo.logout()

        return doc
    
get_data2.execute()
doc = getdata.provenance()
print(doc.get_provn())
print(json.dumps(json.loads(doc.serialize()), indent=4))

eof

KeyboardInterrupt: 

In [52]:
import pandas as pd
path = "C://Users//H//Downloads//monthlyutilitybills.csv"
df = pd.read_csv(path)

In [53]:
updated = pd.DataFrame()
updated['id'] = df['InvoiceID']
updated['TotalConsumption'] = df['TotalConsumption']
updated['StreetAddress'] = df['StreetAddress']
updated.head()

,id,TotalConsumption,StreetAddress
0,6800107,3514.0,5 Drydock Ave
1,6909059,421.0,"Ashmont St., Dorchester Ave., and Talbot Ave. ..."
2,6802634,342.0,Braddock Pk. 40nw. Columbus Ave.
3,6927149,1922.0,D St. and 401
4,6818686,1556.0,Cleveland Cir. and 1922 Beacon St.


In [54]:
from geopy.geocoders import Nominatim

geolocator = Nominatim()
location = geolocator.geocode("5 Drydock Ave")
print(location.latitude, location.longitude)

42.3447184427405 -71.0361999549896


In [94]:
def findcoords(row):
    x = row['StreetAddress']
    location = geolocator.geocode(x, timeout=10)
    if location is None:
        return None
    if 'Boston' in str(location):
        return (location.latitude, location.longitude)
    else:
        return None

updated['Coordinates'] = updated[0:318000].apply(findcoords, axis=1)

# for i in range(0, 300, 100):
#     updated['Coordinates'] = updated[i: i + 100].apply(findcoords, axis=1)

KeyboardInterrupt: 

In [91]:
updated

,id,TotalConsumption,StreetAddress,Coordinates
0,6800107,3514.00,5 Drydock Ave,"(42.3447184427405, -71.0361999549896)"
1,6909059,421.00,"Ashmont St., Dorchester Ave., and Talbot Ave. ...",None
2,6802634,342.00,Braddock Pk. 40nw. Columbus Ave.,None
3,6927149,1922.00,D St. and 401,None
4,6818686,1556.00,Cleveland Cir. and 1922 Beacon St.,"(42.3362284, -71.1489363)"
5,6778304,2935.00,Queensbury St. Yawkey Way,None
6,10336391,191.00,"Cambridge St., New Sudbury St., and Somerset St.",None
7,6856372,1317.00,1535 Washington St,None
8,7045705,5000.00,112 Paris St,"(42.3728416363636, -71.0372819090909)"
9,6961933,209.00,Boylston St. opp. 1270,None


In [51]:
import numpy as np

updated = updated.replace(to_replace='None', value=np.nan).dropna()
updated

,id,TotalConsumption,StreetAddress,Coordinates
0,6800107.0,3514.0,5 Drydock Ave,"(42.3447184427405, -71.0361999549896)"
3,6927149.0,1922.0,D St. and 401,"(45.366683, -122.583296)"
4,6818686.0,1556.0,Cleveland Cir. and 1922 Beacon St.,"(42.3362284, -71.1489363)"
7,6856372.0,1317.0,1535 Washington St,"(47.2056906868687, -122.23497369697)"
8,7045705.0,5000.0,112 Paris St,"(42.3728416363636, -71.0372819090909)"
12,6797023.0,1746.0,Atlantic Ave. C.A.,"(36.9645094, -122.0032718)"
13,10866121.0,96.0,Dorchester Ave. @ Kimball St.,"(42.304142, -71.059128)"
